In [ ]:
import os
import sys
import time
import glob
import shutil
import subprocess
from urllib.parse import urlparse

#检查是否使用GPU
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
!git config --global advice.detachedHead false
def check_gpu():
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) > 0:
        print("GPU is available")
    else:
        sys.exit("\n没有使用GPU，请在代码执行程序-更改运行时类型-设置为GPU！\n如果不能使用GPU，建议更换账号！")
check_gpu()
#运行代码时,打印错误信息
def run_code(code):
    try:
        output = subprocess.check_output(code, stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:
        print(f"run code '{e.cmd}' failed with error:")
        print(e.output.decode())

#@markdown ##<font color="#11659a"> 0️⃣模式选择：
#@markdown #####<font color="#2b73af"> colab是将Fooocus安装到colab；drive是将Fooocus安装到google云盘或启动云盘里的sd。 
sd_path = "colab"  # @param ["colab", "drive"]
if sd_path == "colab":
    sd_dir = f"/content/"
    print('将SD安装在colab磁盘中。')
else:
    sd_dir = f"/content/drive/MyDrive/"
    print('将SD安装在google硬盘中。')

#@markdown ####<font color="#11659a"> 版本选择：
#@markdown #####<font color="#2c9678"> Fooocus是官方版；Fooocus-MRE是进阶版。 
foocus_versions = "Fooocus-MRE" #@param ["Fooocus","Fooocus-MRE"]
if foocus_versions == "Fooocus":
    sd_name = "Fooocus"
else:
    sd_name = "Fooocus-MRE"

#@markdown #####<font color="#2c9678">☑链接谷歌云盘；☐不使用谷歌云盘。
use_drive = False  #@param {type:"boolean"}
if sd_path == "drive" or use_drive:
    if not os.path.exists('/content/drive'):
        from google.colab import drive
        drive.mount('/content/drive')
        print('Google Drive 已挂载成功！')

%cd {sd_dir}
webui_dir = sd_dir + sd_name
#@markdown #####<font color="#2c9678">☑更新云盘Fooocus；☐不使用更新。
update_sd = False  #@param {type:"boolean"}
if os.path.exists(webui_dir) and update_sd:
    !git -C {webui_dir} reset --hard && git -C {webui_dir} checkout master && sleep 1 && rm {webui_dir}/webui.sh && git -C {webui_dir} pull
    print('Fooocus,更新完成！')
else:
    if foocus_versions == "Fooocus":
        !git clone -q --branch master https://github.com/lllyasviel/Fooocus
    else:
        !git clone -q --branch moonride-main https://github.com/MoonRide303/Fooocus-MRE
        !cp {webui_dir}/settings-no-refiner.json {webui_dir}/settings.json
    print('Fooocus,克隆完成！')

print('正在安装 Fooocus 运行环境!')
run_code('apt-get -y update -qq')
run_code('apt-get -y install -qq aria2 git')
if foocus_versions == "Fooocus":
    run_code('pip install torchsde==0.2.5 einops==0.4.1 transformers==4.30.2 safetensors==0.3.1 accelerate==0.21.0')
    run_code('pip install pytorch_lightning==1.9.4 omegaconf==2.2.3 gradio==3.39.0 xformers==0.0.20 triton==2.0.0 pygit2==1.12.2')
elif foocus_versions =="Fooocus-MRE":
    run_code('pip install torchsde==0.2.5 einops==0.4.1 transformers==4.30.2 safetensors==0.3.1 accelerate==0.21.0 aiohttp==3.8.5')
    run_code('pip install pytorch_lightning==1.9.4 omegaconf==2.2.3 gradio==3.41.2 xformers==0.0.21 triton==2.0.0 pygit2==1.12.2 fastapi==0.94.0')   
print('运行环境，安装成功!')

#@markdown ---
#@markdown ##<font color="#11659a"> 1️⃣初始模型下载：
#@markdown #####<font color="#2c9678">🔅☑把主模型下载到colab；☐把模型下载到云盘。
ckpt_to_colab = True  #@param {type:"boolean"}

if ckpt_to_colab or sd_path == "colab":
    ckpt_dir = f"/content/{sd_name}/models/checkpoints"
else:
    ckpt_dir = f"/content/drive/MyDrive/{sd_name}/models/checkpoints"

#@markdown #####<font color="#20894d"> ckpt_downlink 为直接下载链接，ckpt_filename 为模型文件名.
ckpt1 = "SDXL_base" #@param ["SDXL_base","RunDiffusionXL","JuggernautXL","自定义", ""]
ckpt2 = "" #@param ["SDXL1.0_refiner","自定义", ""]
ckpt1_downlink = "" #@param {type:"string"}
ckpt2_downlink = "" #@param {type:"string"}

if ckpt1 == 'SDXL_base':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_base_1.0_0.9vae.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_base_1.0/resolve/main/sd_xl_base_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_base_1.0_0.9vae.safetensors
elif ckpt1 == 'RunDiffusionXL':
    if not os.path.exists(f'{ckpt_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/rundiffusion-xl/resolve/main/rundiffusionXL_beta.safetensors -d {ckpt_dir} -o rundiffusionXL_beta.safetensors
elif ckpt1 == 'JuggernautXL':
    if not os.path.exists(f'{ckpt_dir}/juggernautXL_version1.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/juggernautXL_version1.safetensors -d {ckpt_dir} -o juggernautXL_version1.safetensors
elif ckpt1 == '\u81EA\u5B9A\u4E49':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt1_downlink} -d {ckpt_dir}
else:
    print("暂时不下载主模型！")

if ckpt2 == 'SDXL_refiner':
    if not os.path.exists(f'{ckpt_dir}/sd_xl_refiner_1.0.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/sd_xl_refiner_1.0/resolve/main/sd_xl_refiner_1.0_0.9vae.safetensors -d {ckpt_dir} -o sd_xl_refiner_1.0.safetensors
elif ckpt2 == '\u81EA\u5B9A\u4E49':
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {ckpt2_downlink} -d {ckpt_dir}

#@markdown #####<font color="#2c9678">🔅☑把Lora模型下载到colab；☐把模型下载到云盘。
lora_to_colab = True  #@param {type:"boolean"}

if lora_to_colab or sd_path == "colab":
    lora_dir = f"/content/{sd_name}/models/loras"
else:
    lora_dir = f"/content/drive/MyDrive/{sd_name}/models/Lora"

#@markdown #####<font color="#20894d"> lora_downlink 为直接下载链接，lora_filename 为模型文件名.
lora1 = "" #@param ["JuggerCineXL2","自定义", ""]
lora_downlink = "" #@param {type:"string"}

if lora1 == 'JuggerCineXL2':
    if not os.path.exists(f'{lora_dir}/rundiffusionXL_beta.safetensors'):
        !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/ckpt/juggernaut-xl/resolve/main/JuggerCineXL2.safetensors -d {lora_dir} -o JuggerCineXL2.safetensors
elif lora1 == '\u81EA\u5B9A\u4E49':
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {lora_downlink} -d {lora_dir}

# 链接 VAE
vae_dir = f"{webui_dir}/models/embeddings"

# 链接 embeddings
embeddings_dir = f"{webui_dir}/models/embeddings"

# 链接 clip_vision
clip_vision_dir = f"{webui_dir}/models/clip_vision"

# 链接 hypernetwork
hypernetworks_dir = f"{webui_dir}/models/hypernetworks" 

# 链接 scripts

# 链接 ControlNet
#@markdown ####<font color="#2b73af"> 选择ControlNet：
#@markdown #####<font color="#2c9678">🔅☑把ControlNet模型下载到colab；☐把模型下载到云盘。
cn_to_colab = True  #@param {type:"boolean"}

if cn_to_colab or sd_path == "colab":
    cn_dir = f"/content/{sd_name}/models/controlnet"
else:
    cn_dir = f"/content/drive/MyDrive/{sd_name}/models/controlnet"

def download(url, model_dir):
    filename = os.path.basename(urlparse(url).path)
    pth = os.path.abspath(os.path.join(model_dir, filename))
    if not os.path.exists(pth):
        print('Downloading: ' + os.path.basename(url))
        cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M -d {model_dir} -o {filename} {url}"
        subprocess.run(cmd, shell=True, check=True)
    else:
        print(f"[1;32mThe model {filename} already exists[0m")

def cn_down():
    if os.path.exists("CN_models.txt"):
        with open("CN_models.txt", 'r') as f:
            mdllnk = f.read().splitlines()
    for lnk in mdllnk:
        download(lnk, cn_dir)

controlnet = "" # @param ["", "SD15_base", "SD15_full", "SDXL_base", "SDXL_full"]
if controlnet == "SD15_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15base.txt")
    cn_down()
elif controlnet == "SD15_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sd15full.txt")
    cn_down()
elif controlnet == "SDXL_base":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlbase.txt")
    cn_down()
elif controlnet == "SDXL_full":
    os.system(f"curl -o CN_models.txt https://raw.githubusercontent.com/Van-wise/sd-colab/main/sdxl/cnDL_sdxlfull.txt")
    cn_down()
else:    
    print("不使用 ControlNet、或已使用快捷方式。")

#@markdown ##<font color="#11659a"> MOD!
import pandas as pd
from google.colab import files, drive
#@markdown #####<font color="#2c9678">🔅☑启用MOD功能；☐不使用MOD。
enable_mod = False #@param {type:"boolean"} 
mod_name = "mod.xlsx"   #@param {type:"string"}
mod_link = ""   #@param {type:"string"} 

def get_mod():
    if mod_link:
        filename = os.path.basename(mod_link)
        if not filename.endswith('.xlsx'):
            print("下载链接错误!")
            return None
        !wget -q -T 10 {mod_link} -O {filename}
        mod_file = f"/content/{filename}"
        return mod_file 

    else:
        while True:
            try:
                uploaded = files.upload()
                if not uploaded:
                    mod_file = None
                    break
                mod_file = os.path.join(os.getcwd(), list(uploaded.keys())[0])

                if not mod_file.endswith('.xlsx'):
                    raise ValueError("文件格式错误 .xlsx ")
                break

            except (ValueError, Exception) as e:
                print(e)
                print("请重新上传文件。")

        return mod_file

def use_mod():
    df = pd.read_excel(mod_file)

    for index, row in df.iterrows():
        file_type = row['类型']
        file_name = row['名称']
        download_link = row['下载地址']

        if file_type == 'CKPT':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {ckpt_dir} -o {file_name}
        elif file_type == 'Lora':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {lora_dir} -o {file_name}
        elif file_type == 'VAE':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {vae_dir} -o {file_name}
        elif file_type == 'Extensions':
            repo_name = download_link.split('/')[-1]
            !git clone {download_link} {webui_dir}/extensions/{repo_name}
        elif file_type == 'Embeddings':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {embeddings_dir} -o {file_name}
        elif file_type == 'Scripts':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {webui_dir}/scripts -o {file_name}
        elif file_type == 'Controlnet':
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {download_link} -d {cn_dir} -o {file_name}
        elif file_type == 'Comfyui_Nodes':
            nodes_name = download_link.split('/')[-1]
            !git clone {download_link} {webui_dir}/extensions/{nodes_name}

if enable_mod:
    mod_file = get_mod()
    if mod_file:
        use_mod()

#@markdown---
#@markdown ####<font color="#2b73af"> 🌐☑使用云盘模型；☐不使用云盘模型。
use_drive_model = False  #@param {type:"boolean"}
#@markdown #####<font color="#2b73af"> 两种方法：cp（从云盘复制到colab）；ln（添加云盘软链接到colab）
content_way = "\u94FE\u63A5ln" #@param ["\u590D\u5236cp", "\u94FE\u63A5ln"]
if use_drive and use_drive_model:
    gckpt_dir = f"/content/drive/MyDrive/{sd_name}/models/checkpoints"
    gLora_dir = f"/content/drive/MyDrive/{sd_name}/models/Loras"
    gVAE_dir = f"/content/drive/MyDrive/{sd_name}/models/vae"
    gembeddings_dir = f"/content/drive/MyDrive/{sd_name}/embeddings"
    gcn_dir = f"/content/drive/MyDrive/{sd_name}/extensions/models/controlnet"
    goutputs_dir = f"/content/drive/MyDrive/{sd_name}/outputs"
    !mkdir -p {gLora_dir} {gVAE_dir} {gembeddings_dir} {gcn_dir} {goutputs_dir} {gckpt_dir}

    if content_way == "\u590D\u5236cp":
        !cp -r {gckpt_dir} {ckpt_dir}
        !cp -r {gLora_dir} {Lora_dir}
        !cp -r {gVAE_dir} {VAE_dir}
        !cp -r {gembeddings_dir} {embeddings_dir}
        !cp -r {gcn_dir} {cn_dir}
    else:
        !ln -sf {gckpt_dir} {ckpt_dir}
        !ln -sf {gLora_dir} {Lora_dir}
        !ln -sf {gVAE_dir} {VAE_dir}
        !ln -sf {gembeddings_dir} {embeddings_dir}
        !ln -sf {gcn_dir} {cn_dir}

#@markdown---
#@markdown ##<font color="#11659a">▶️ Start Fooocus
%cd {webui_dir}
!python launch.py --share
